<center>
<div class="h1">Info 114: Introduction to Data Science</div>
<div class="h1">Homework 3: from VI-ME-BA-BAR to POM</div>
</center>

# Part 3: Project selection
You will find below some code that was written to prepare the class.
We will go more in depth over these questions in later classes.
This is a "preview" to help you select your project.

Our problem seems rather complex. Non-linear methods perform best.
Unfortunately, these methods are hard to understand.
To make progress, we must try to visualize our data again.

In this worksheet, we use as example the data of `CS_data.csv`, after standardization.

## Setup: import code an load data

In [ ]:
import os, re
from glob import glob as ls
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()
from PIL import Image
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [ ]:
code_dir = './code'
from sys import path; path.append(code_dir); 
from utilities import *
from my_utilities import *

In [ ]:
data_dir = './mini-dataset/'
data_df = standardize_df(pd.read_csv(data_dir + 'CS_data.csv'))

## Heatmap

Since it is very important to visualize data, we show below a heatmap of the data in which the lignes and columns have been re-arranged according to their resemblance. This is an "unsupervised learning" visualization method, so we omit the last column.

In [ ]:
data_df.head()

In [ ]:
df = data_df.iloc[:, :-1]
heatmap(df, 'average', 'single', 'euclidean', 'euclidean', 'coolwarm')

For comparison, we show what happens if we randomize completely the order of the values.

In [ ]:
df2 = shuffle(df)
heatmap(df2, 'average', 'single', 'euclidean', 'euclidean', 'coolwarm')

## Correlation


We display the correlation matrix of `data_df`, following <a href="https://stackoverflow.com/questions/29432629/plot-correlation-matrix-using-pandas">this page</a>.

In [ ]:
corr = data_df.corr()
corr.style.background_gradient(cmap='coolwarm')

We can also use the hierarchical clustering.

In [ ]:
heatmap(corr, 'average', 'average', 'euclidean', 'euclidean', 'coolwarm', default_window_hight = 15, default_window_width = 15)

You can look for the most correlated or anti-correlated feature/variable.

In [ ]:
np.fill_diagonal(corr.values, 0)
print('Most correlated: ' + corr.fruit.argmax())
print('Most anti-correlated: ' + corr.fruit.argmin())

Are you surprised? I was surprised. In the first toy dataset we played with, apples were correlated with red. This is no longer the case. You can check out the dataset and understand why. In this new dataset, most apples are gree. Many bananas are yellow and some are green.

In [ ]:
img = get_image(data_dir + 'all_data.png')
img

The dataset is not completely well balanced, so the Pearson correlation coefficient may not be the best way to identify the most informative features. Let's see whether S2N makes a difference.

In [ ]:
s2n_coeff = s2n(data_df)
print('Largest s2n: ' + s2n_coeff.feat.argmax())
print('Smallest s2n: ' + s2n_coeff.feat.argmin())

We get the same features with S2N and the Pearson correlation coefficient.

## Feature selection
We are going to perform some simple-minded feature selection with the Pearson correlation coefficient. This will allow us to do some data visualization.

Let us first sort all features by the absolute value of the Pearson correlation coefficient. Indeed, variables are informative no matter whether they are correlated or anti-correlated (since it suffices to multiply them by -1 to change the correlation direction).

In [ ]:
# Sort by correlation coefficient
sval = corr['fruit'][:-1].abs().sort_values(ascending=False)
ranked_columns = sval.index.values
print(ranked_columns) 

We notice that the features that we had constructed in previous lessons 'R-(G+B)/2' and 'W/H' come in the 5 top most informative features. But there are others. Let us make all scatter plots of pairs of features for the 5 top ranked features.

In [ ]:
fruit_name = ['Banana', 'Apple']
fruit_list = [fruit_name[int((i+1)/2)] for i in data_df["fruit"].tolist()]

In [ ]:
col_selected = ranked_columns[0:5]
df_new = pd.DataFrame.copy(data_df)
df_new = df_new[col_selected]
df_new['fruit'] = fruit_list
g = sns.pairplot(df_new, hue="fruit", markers=["o", "s"], diag_kind="hist")

It is interesting to see how many features are needed to obtain nearly as good performance as with all the features.

In [ ]:
from sklearn.metrics import balanced_accuracy_score as sklearn_metric
sklearn_model = KNeighborsClassifier(n_neighbors=3)
feat_lc_df = feature_learning_curve(data_df, sklearn_model, sklearn_metric)

In [ ]:
#feat_lc_df[['perf_tr', 'perf_te']].plot()
plt.errorbar(feat_lc_df.index+1, feat_lc_df['perf_tr'], yerr=feat_lc_df['std_tr'], label='Training set')
plt.errorbar(feat_lc_df.index+1, feat_lc_df['perf_te'], yerr=feat_lc_df['std_te'], label='Test set')
plt.xticks(np.arange(1, 22, 1)) 
plt.xlabel('Number of features')
plt.ylabel(sklearn_metric.__name__)
plt.legend(loc='lower right')

We see the, with 5 features, it is about as good as it gets, given the error bars.

We can then investigate all pairs of features among the top 5 to see which pair is best.

In [ ]:
range(5)

In [ ]:
best_perf = -1
std_perf = -1
best_i = 0
best_j = 0
for i in np.arange(5): 
    for j in np.arange(i+1,5): 
        df = data_df[[ranked_columns[i], ranked_columns[j], 'fruit']]
        p_tr, s_tr, p_te, s_te = df_cross_validate(df, sklearn_model, sklearn_metric)
        if p_te > best_perf: 
            best_perf = p_te
            std_perf = s_te
            best_i = i
            best_j = i
            
metric_name = sklearn_metric.__name__.upper()
print('BEST PAIR: {}, {}'.format(best_i, best_j))
print("AVERAGE TEST {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_te, s_te))

Not too surprisingly the first two features are best. We can also run a different kind of feature selection, but the results are very similar.

In [ ]:
# From https://scikit-learn.org/stable/modules/feature_selection.html
# 1.13.4.2. Tree-based feature selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
X = data_df.iloc[:, :-1].to_numpy()
Y = data_df.iloc[:, -1].to_numpy()
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, Y)
ranked_columns = data_df.columns[np.argsort(-clf.feature_importances_)]
print(ranked_columns)

## Exploring the metadata
For those of you who will chose the project on how to find and correct bias in data, part of your work will be to analyze the metadata: can you predict for example "apple" vs. "banana" based on the meta-features? This would indicate some confounding factor.

In [ ]:
metadata = pd.read_csv(os.path.join(data_dir, 'metadata.csv')) #, index_col='Num')
metadata.head()

In [ ]:
# Select images with a single apple or banana
is_apple  = metadata['Fruit'] == 'APPLE'
is_banana = metadata['Fruit'] == 'BANANA'
count_one = metadata['Count'] == '1-ONE'
apple_subset = metadata[is_apple & count_one]
banana_subset = metadata[is_banana & count_one]
print('Apples: {}'.format(apple_subset.shape))
print('Bananas: {}'.format(banana_subset.shape))

In [ ]:
# Only two variables are numerical, the others would have to be converted.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html may not be the best solution
# Read also https://www.kaggle.com/c/titanic/discussion/5379
# See also https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder

metadata.describe()

In [ ]:
series = ['Num', 'GPSAltitude', 'SubsecTimeDigitized']
metadata_subset = apple_subset.copy()[series]
metadata_subset = metadata_subset.append(banana_subset.copy()[series])
na = len(apple_subset)
nb = len(banana_subset)
Y = np.append(np.ones([na, 1]), -1*np.ones([nb, 1]))
metadata_subset['fruit'] = Y
print(df.shape)
metadata_subset.head()

In [ ]:
# Rappel de la question 2.1
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score as sklearn_metric
sklearn_model = KNeighborsClassifier(n_neighbors=1)

In [ ]:
naval = check_na(metadata_subset)

In [ ]:
metadata_subset[metadata_subset.isna()]=0

In [ ]:
#metadata_subset.GPSAltitude[895]=0

In [ ]:
check_na(metadata_subset)

In [ ]:
p_tr, s_tr, p_te, s_te = df_cross_validate(metadata_subset, sklearn_model, sklearn_metric, n=10, verbose=False)
metric_name = sklearn_metric.__name__.upper()
print("AVERAGE TRAINING {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_tr, s_tr))
print("AVERAGE TEST {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_te, s_te))

In [ ]:
corr = metadata_subset.corr()
corr.style.background_gradient(cmap='coolwarm')

## Dimensionality reduction

Rather than doing feature selection, another avenue is to use feature transforms to reduce dimensionality, typically find the "principal directions"(directions of largest variance). This is achieved with SVD (the same algorithm we used last week to find the aspect ratio of an elongated object). There again I used my search engine and typed the keywords "pandas svd". I found a nice tutorial on [this page](https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/) and a step-by-step procedure on [this page](https://cmdlinetips.com/2019/05/singular-value-decomposition-svd-in-python/). Can you transform the original data frame into a data frame with only 2 features (the two first principal directions)?

<u>Question 1:</u> Create a dataframe called `df_scaled` containing the standardized columns, except the last one (tip: use `drop` to eliminate the last column).

In [ ]:
### BEGIN SOLUTION
df = pd.read_csv(os.path.join(data_dir, 'CS_data.csv'))
df_bare = df.drop(columns=['fruit'])
df_scaled = (df_bare-df_bare.mean())/df_bare.std()
df_scaled.head()
### END SOLUTION

<u>Question 2:</u> Perform a singular value decomposition of `df_scaled` and call the resulting matrices u, s, v.

In [ ]:
### BEGIN SOLUTION
u, s, v = np.linalg.svd(df_scaled, full_matrices=True)
print('U {}'.format(u.shape))
print('S {}'.format(s.shape))
print('V {}'.format(v.shape))
### END SOLUTION

<u>Question 3:</u> Make a scree plot of the eigen values (square of the singular values). Save the plot in file 'svd_scree_plot.png'.

In [ ]:
### BEGIN SOLUTION
var_explained = np.round(s**2/np.sum(s**2), decimals=3)
sns.barplot(x=list(range(1,len(var_explained)+1)),
            y=var_explained, color="limegreen")
plt.xlabel('SVs', fontsize=16)
plt.ylabel('Percent Variance Explained', fontsize=16)
plt.savefig('svd_scree_plot.png',dpi=100)
### END SOLUTION

<u>Question 4:</u> Create a new dataframe `svd_df` with the two singular values as columns and the fruit type as index.

In [ ]:
### BEGIN SOLUTION
labels= ['SV'+str(i) for i in range(1,4)]
fruit_name = ['Banana', 'Apple']
fruit_list = [fruit_name[int((i+1)/2)] for i in df["fruit"].tolist()]
svd_df = pd.DataFrame(u[:,0:3], columns=labels)
svd_df['fruit'] = fruit_list
svd_df.head()
### END SOLUTION

<u>Question 5:</u> Make pairwise scatter plots of the three first singular values.

In [ ]:
### BEGIN SOLUTION
g = sns.pairplot(svd_df, hue="fruit", markers=["o", "s"], diag_kind="hist")
### END SOLUTION

<u>Question 6:</u> Compute the performances obtained with the 3 nearest neighbor method using the first 3 singular values.

In [ ]:
### BEGIN SOLUTION
svd_df['fruit'] = df.iloc[:, -1].to_numpy()
p_tr, s_tr, p_te, s_te = df_cross_validate(svd_df, sklearn_model, sklearn_metric)
metric_name = sklearn_metric.__name__.upper()
print("AVERAGE TRAINING {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_tr, s_tr))
print("AVERAGE TEST {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_te, s_te))
### END SOLUTION